<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/SQL/COLABs%20/ABC_Gleb_Mikhailov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
import numpy as np
from google.colab import userdata
SQLALCHEMY_SILENCE_UBER_WARNING=1

In [ ]:
engine = create_engine(userdata.get('elefantSQL'))

In [ ]:
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [ ]:
sql = """
-- Сгруппируем товары по количесвтву продаж
with product_grouped as (
select
  dr_ndrugs, sum(dr_kol) as amounts,
  sum((dr_croz - dr_czak) * dr_kol - dr_sdisc) as pdofit_sum,
  sum(dr_croz * dr_kol - dr_sdisc) as revenue_sum
from
  sales
group by dr_ndrugs
)


-- Выполним ABC анализ по количеству (A-B-C  => 80-15-5%)
select product_grouped.dr_ndrugs,
case
    when sum(product_grouped.amounts) over (order by amounts desc) <= (select sum(dr_kol) * 0.8 from sales) then 'A'
    when sum(product_grouped.amounts) over (order by amounts desc) <= (select sum(dr_kol) * 0.95 from sales) then 'B'
    else 'C'
end as group_amount,
case
    when sum(product_grouped.pdofit_sum) over (order by pdofit_sum desc) <= (select sum((dr_croz - dr_czak) * dr_kol - dr_sdisc)* 0.8 from sales) then 'A'
    when sum(product_grouped.pdofit_sum) over (order by pdofit_sum desc) <= (select sum((dr_croz - dr_czak) * dr_kol - dr_sdisc)* 0.95 from sales) then 'B'
    else 'C'
end as group_profit,
case
    when sum(product_grouped.revenue_sum) over (order by revenue_sum desc) <= (select sum(dr_croz * dr_kol - dr_sdisc)* 0.8 from sales) then 'A'
    when sum(product_grouped.revenue_sum) over (order by revenue_sum desc) <= (select sum(dr_croz * dr_kol - dr_sdisc)* 0.95 from sales) then 'B'
    else 'C'
end as group_revenue

from product_grouped
limit 1000
"""

In [ ]:
select(sql)